# Generate Consistent Images for the CNN

*The functions `multiply_image` and `multiply_image_backward` are based on the code Chris Birchall used to [find Wally](https://tech.ovoenergy.com/cheating-at-wheres-wally/).*

*Using [this resource](https://note.nkmk.me/en/python-opencv-pillow-image-size/) to leverage `opencv` functionality to automatically-detect image size.*

*Using [imagej](https://imagej.nih.gov/ij/) to identify the location of each kithara and write out x-y bounding box coordinates for each.*

In [1]:
# import necessary libraries
import cv2
import glob
import os.path
import pandas as pd
import numpy as np

### Test out extracting the image size

In [2]:
# # get the height, width, and color of an image
# # read in the image
# test_img = cv2.imread("../data/test/IMAG0009.jpg")
# # assign the results to variables y (height), x (width), and c (color)
# y, x, c = test_img.shape

# # # alternatively, select specific values in the array
# # x_max = test_img.shape[1]
# # y_max = test_img.shape[0]

# # show the type of the variable created and return some stats
# print(type(test_img))
# print(test_img.shape)
# print(type(test_img.shape))
# print(y)
# print(x)
# print(c)

### Test out extracting the kithara size and location 

In [3]:
# # read in the csv with the kithara location data
# test_kithara = pd.read_csv('../data/coordinates/IMAG0009.csv').iloc[:, 1:]
# test_kithara

In [4]:
# # get the min and max x and y coordinates, plus the x distance and y distance
# kx_min = test_kithara.x_top_left.values[0]
# kx_max = test_kithara.x_bottom_right.values[0]
# ky_min = test_kithara.y_top_left.values[0]
# ky_max = test_kithara.y_bottom_right.values[0]
# kx_size = kx_max - kx_min
# ky_size = ky_max - ky_min
# k_max = np.where(kx_size > ky_size, kx_size, ky_size)
# print('width (x): ' + kx_size.astype(str))
# print('height (y): ' + ky_size.astype(str))
# print('max size: ' + k_max.astype(str))

In [5]:
# k_max = np.where(kx_size > ky_size, kx_size, ky_size).item(0)
# k_max

In [6]:
# k_inc = int(k_max/2)
# print(type(k_inc))

### Write a couple of functions to cut up each image into multiple equally-sized, square chunks

*Since images are not necessarily going to be evenly-dissectable based on the size of the kithara for each, I'm taking two tactics to ensure full-coverage:*
* Chopping each image up starting from the top left (`multiply_image`) and bottom right (`multiply_image_backward`) corners
* Shifting the x and y start coordinates by 1/2 of the kithara image size at a time

*To tag whether or not an image includes a full or partial kithara, I'm doing the following:*
* Taking the x-y coordinates of the kithara image location (generated using `imagej`) and, if the image falls within those coordinates, tagging it as a kithara

*To clean up the data generated, I'm doing the following:*
* Deleting any non-square image from the folder with the cropped images
* Resizing all of the square images to the same dimensions

In [7]:
# for each image in the folder
# get the height and width of each image
# then iterate over the image and create 1000 x 1000 pixel squares
def multiply_image(name):
    # read in the image
    img = cv2.imread("../data/test/{}.jpg".format(name))
    # get the size of the image
    y_max, x_max, _ = img.shape  
    # get the coordinates of the kithara
    kithara = pd.read_csv('../data/coordinates/{}.csv'.format(name)).iloc[:, 1:]
    kx_min = kithara.x_top_left.values[0]
    kx_max = kithara.x_bottom_right.values[0]
    ky_min = kithara.y_top_left.values[0]
    ky_max = kithara.y_bottom_right.values[0]
    kx_size = kx_max - kx_min
    ky_size = ky_max - ky_min
    # set the image size based on the larger of x or y for the kithara
    k_max = np.where(kx_size > ky_size, kx_size, ky_size).item(0)
    # set the increment to 1/2 the size of the larger of x or y for the kithara
    k_inc = int(k_max/2)
    # incrementally move forward, starting at the top L corner, by 1/2 the largest dimension of the kithara size
    # get a bunch of square images that are the size of the kithara
    for y in range(0, y_max, k_inc):
        start_y = y
        end_y = y + k_max 
        for x in range(0, x_max, k_inc):
            start_x = x
            end_x = x + k_max
            # set the image coordinates
            cropped_img = img[start_y:end_y, start_x:end_x]
            # crop the image
            cv2.imwrite("../data/cropped_images/{}-{}-{}.png".format(name, y, x), cropped_img)

In [8]:
# go over the image but in reverse and create pixel squares that are the size of the longest of the kithara dimensions
def multiply_image_backward(name):
    # read in the image
    img = cv2.imread("../data/test/{}.jpg".format(name))
    # get the size of the image
    y_max, x_max, _ = img.shape
    # get the coordinates of the kithara
    kithara = pd.read_csv('../data/coordinates/{}.csv'.format(name)).iloc[:, 1:]
    kx_min = kithara.x_top_left.values[0]
    kx_max = kithara.x_bottom_right.values[0]
    ky_min = kithara.y_top_left.values[0]
    ky_max = kithara.y_bottom_right.values[0]
    kx_size = kx_max - kx_min
    ky_size = ky_max - ky_min
    # set the image size based on the larger of x or y for the kithara
    k_max = np.where(kx_size > ky_size, kx_size, ky_size).item(0)
    # set the increment to 1/2 the size of the larger of x or y for the kithara
    k_inc = int(k_max/2)
    # incrementally move backward, starting at the bottom R corner, by 1/2 the largest dimension of the kithara size
    # get a bunch of square images that are the size of the kithara
    for y in range(y_max, 0, -k_inc):
        if y >= k_max:
            start_y = y - k_max
            end_y = y
        else:
            start_y = 0
            end_y = y
        for x in range(x_max, 0, -k_inc):
            if x >= k_max:
                start_x = x - k_max
                end_x = x
            else:
                start_x = 0
                end_x = x
            # set the image coordinates
            cropped_img = img[start_y:end_y, start_x:end_x]
            # crop the image
            cv2.imwrite("../data/cropped_images/back-{}-{}-{}.png".format(name, end_y, end_x), cropped_img)

In [9]:
def get_kithara(name):
    img = cv2.imread("../data/test/{}.jpg".format(name))
    kithara = pd.read_csv('../data/coordinates/{}.csv'.format(name)).iloc[:, 1:]
    kx_min = kithara.x_top_left.values[0]
    kx_max = kithara.x_bottom_right.values[0]
    ky_min = kithara.y_top_left.values[0]
    ky_max = kithara.y_bottom_right.values[0]
    kx_size = kx_max - kx_min
    ky_size = ky_max - ky_min
    k_max = np.where(kx_size > ky_size, kx_size, ky_size).item(0)
    if kx_size < ky_size:
        x_start = int(kx_min - ((ky_size - kx_size)/2))
        x_end = int(kx_max + ((ky_size - kx_size)/2))
        y_start = ky_min
        y_end = ky_max
    elif ky_size < kx_size:
        y_start = int(ky_min - ((kx_size - ky_size)/2))
        y_end = int(ky_max + ((kx_size - ky_size)/2))
        x_start = kx_min
        x_end = kx_max
    else:
        x_start = kx_min
        x_end = kx_max
        y_start = ky_min
        y_end = ky_max
    kithara_img = img[y_start:y_end, x_start:x_end]
    cv2.imwrite("../data/kitharai/{}-kithara.png".format(name), kithara_img)

In [10]:
# run the functions over the image folder    
for path in glob.glob("../data/test/*.jpg"):
    name = os.path.splitext(os.path.basename(path))[0]
    get_kithara(name)
    multiply_image(name)
    multiply_image_backward(name)

### Next Steps
* Tag images that intersect the boxes with the kithara in it
* Get rid of incorrectly-sized images
* Refactor the functions above to prevent unnecessary duplication of work
* Test on a larger dataset